# Non-Smooth Optimization Using pyOpt and AlgoPy

This notebook uses package [`pyOpt`](http://www.pyopt.org/index.html) for optimization, and package [`algopy`](https://pythonhosted.org/algopy/) to evaluate gradients (subgradients) for non-smooth functions. Specifically, we use [`SOLVOPT`](http://www.pyopt.org/reference/optimizers.solvopt.html) minimizer in `pyOpt` to implement a modified version of Shor’s r–algorithm on non-smooth functions.  

`pyOpt` has the following dependencies: Python 2.4+, Numpy 1.0+, Swig 1.3+, c/FORTRAN compiler (compatible with f2py). Here Python 2.7 and numpy 1.10.1 are used (newest version of `numpy` may not work on `pyOpt` for unsolved compatibility issues). See `pyOpt` [documentation](http://www.pyopt.org/install.html#installation) for details on installation.

In [1]:
import sys, numpy

print sys.version
print numpy.version.version

2.7.13 |Continuum Analytics, Inc.| (default, Dec 20 2016, 23:05:08) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
1.10.1


In [2]:
import algopy
import numpy as np
import pyOpt
from pyOpt import Optimization
from pyOpt import SOLVOPT

## Implementing Automatic Differentiation Using AlgoPy

In [3]:
class AD():

    def __init__(self, f):
        self.f = f     

    # Forward Mode Derivative Evaluations
    def grad_forward(self, x):
        x = algopy.UTPM.init_jacobian(x)
        return algopy.UTPM.extract_jacobian(self.f(x))
    
    def hess_forward(self, x):
        x = algopy.UTPM.init_hessian(x)
        return algopy.UTPM.extract_hessian(len(x), self.f(x))

    def hess_vector_forward(self, x, v):
        x = algopy.UTPM.init_hess_vec(x, v)
        return algopy.UTPM.extract_hess_vec(len(x), self.f(x))
    
    # Reverse Mode Derivative Evaluations
    def trace_eval_f(self, x):
        cg = algopy.CGraph()
        x = algopy.Function(x)
        y = self.f(x)
        cg.trace_off()
        cg.independentFunctionList = [x]
        cg.dependentFunctionList = [y]
        self.cg = cg

    def grad_reverse(self, x):
        self.trace_eval_f(x)
        return self.cg.gradient(x)

    def hess_reverse(self, x):
        self.trace_eval_f(x)
        return self.cg.hessian(x)
    
    def hess_vector_reverse(self, x, v):
        self.trace_eval_f(x)
        return self.cg.hess_vec(x,v)

## Non-Smooth Test Function: Absolute Value Function

We want to optimize function $f(\mathbf{x})=\sum_{i=1}^{10}|{x_i}|$ where its minimum locates at $\mathbf{x}=\mathbf{0}$. Subgradients are computed through `AlgoPy`, and they are provided as user-defined gradient function to the minimizer. Alternatively, we can also compute subgradients using numerical differentiation by setting `sens_type='FD'` in function `solveopt`, though finite differences may not provide machine level precision as does automatic differentiation.

In [4]:
def sum_abs(x):
    return algopy.sum(algopy.absolute(x))

def objfunc(x):
    f = sum_abs(x)
    g = []
    fail = 0
    return f, g, fail

ad = AD(sum_abs)

def gradfunc(x, f, g):
    g_obj = ad.grad_reverse(x) # Use reverse mode to evaluate subgradient (we can also use forward mode)
    g_obj = list(g_obj)
    
    g_con = []
    fail = 0
    return g_obj, g_con, fail

# Optimization setup
opt_prob = Optimization('Sum of N Absolute Value Functions', objfunc)
opt_prob.addVarGroup('x', 10, type='c', value=np.arange(10, dtype=np.float), lower=-np.inf, upper=np.inf)
opt_prob.addObj('f')
print opt_prob

# Call SOLVOPT minimizer to implement Shor's r-algorithm
solvopt = SOLVOPT()
solvopt.setOption('iprint', -1)
solvopt(opt_prob, sens_type=gradfunc)
print opt_prob.solution(0)


Optimization Problem -- Sum of N Absolute Value Functions

        Objective Function: objfunc

    Objectives:
        Name        Value        Optimum
	     f               0             0

	Variables (c - continuous, i - integer, d - discrete):
        Name    Type       Value       Lower Bound  Upper Bound
	    x_0       c	      0.000000           -inf          inf 
	    x_1       c	      1.000000           -inf          inf 
	    x_2       c	      2.000000           -inf          inf 
	    x_3       c	      3.000000           -inf          inf 
	    x_4       c	      4.000000           -inf          inf 
	    x_5       c	      5.000000           -inf          inf 
	    x_6       c	      6.000000           -inf          inf 
	    x_7       c	      7.000000           -inf          inf 
	    x_8       c	      8.000000           -inf          inf 
	    x_9       c	      9.000000           -inf          inf 


SOLVOPT Solution to Sum of N Absolute Value Functions

        Objective Fu

## Non-Smooth Test Function: Shor's Piece-Wise Quadratic Function

Let's minimize a more complicated function: Shor's piece-wise quadratic function. The unconstrained optimization problem is defined as,

$$
\begin{align}
    \min_{\mathbf{x}}\big\{f(\mathbf{x}):x \in\ {\rm I\!R}^n \big\}
\end{align}
$$

where

$$
f(\mathbf{x})=\max{\big\{ \phi_i(\mathbf{x}): i=1,2,\dots,m \big\}}, \quad \phi_i(\mathbf{x})=b_i\sum_{j=1}^{n}(x_j-a_{ij})^2, \mathbf{x}=(x_1, \dots, x_n)
$$

and 

$$
\mathbf{b}=(b_i) \text{ is a given m vector, } \mathbf{A}=(a_{ij}), i=1,\dots, m, \text{ } j=1,\dots, n, \text{ is a given } m \times n \text{ matrix}
$$


In [5]:
def shor(x):
    A = np.array([[0, 0, 0, 0, 0],
                  [2, 1, 1, 1, 3],
                  [1, 2, 1, 1, 2],
                  [1, 4, 1, 2, 2],
                  [3, 2, 1, 0, 1],
                  [0, 2, 1, 0, 1],
                  [1, 1, 1, 1, 1],
                  [1, 0, 1, 2, 1],
                  [0, 0, 2, 1, 0],
                  [1, 1, 2, 0, 0]])
    b = np.array([1, 5, 10, 2, 4, 3, 1.7, 2.5, 6, 4.5])
    res = 0
    for k in range(b.size):
        temp = b[k] * algopy.sum((x - A[k, :])**2)
        res = 0.5 * (temp + res + algopy.absolute(temp - res)) # max(x, y) = (x+y+abs(x-y)) / 2
    return res

def objfunc(x):
    f = shor(x)
    g = []
    fail = 0
    return f, g, fail

ad = AD(shor)

def gradfunc(x, f, g):
    g_obj = ad.grad_reverse(x) # Use reverse mode to evaluate subgradient (we can also use forward mode)
    g_obj = list(g_obj)
    g_con = []
    fail = 0
    return g_obj, g_con, fail

# Optimization setup
opt_prob = Optimization("Shor's Piece-Wise Quadratic Function", objfunc)
opt_prob.addVarGroup('x', 5, type='c', value=np.array([-1, 1, -1, 1, -1]), lower=-np.inf, upper=np.inf)
opt_prob.addObj('f')
print opt_prob

# Call SOLVOPT minimizer to implement Shor's r-algorithm
solvopt = SOLVOPT()
solvopt.setOption('iprint', -1)
solvopt(opt_prob, sens_type=gradfunc)
print opt_prob.solution(0)


Optimization Problem -- Shor's Piece-Wise Quadratic Function

        Objective Function: objfunc

    Objectives:
        Name        Value        Optimum
	     f               0             0

	Variables (c - continuous, i - integer, d - discrete):
        Name    Type       Value       Lower Bound  Upper Bound
	    x_0       c	     -1.000000           -inf          inf 
	    x_1       c	      1.000000           -inf          inf 
	    x_2       c	     -1.000000           -inf          inf 
	    x_3       c	      1.000000           -inf          inf 
	    x_4       c	     -1.000000           -inf          inf 


SOLVOPT Solution to Shor's Piece-Wise Quadratic Function

        Objective Function: objfunc

    Solution: 
--------------------------------------------------------------------------------
    Total Time:                    2.9959
    Total Function Evaluations:       212
    Sensitivities: <function gradfunc at 0x113660050>

    Objectives:
        Name        Value      